In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import BertTokenizer
import re

In [ ]:
def extrair_texto_da_url(url):
    try:
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            texto = soup.get_text()
            return texto
        else:
            print(f"Erro ao acessar a URL. Código de status: {response.status_code}")
            return None
    except Exception as e:
        print(f"Erro ao acessar a URL: {str(e)}")
        return None

url = 'https://brasil.un.org/pt-br/91863-agenda-2030-para-o-desenvolvimento-sustent%C3%A1vel'

texto_extraido = extrair_texto_da_url(url)

if texto_extraido:
    
    with open('texto_extraido.txt', 'w', encoding='utf-8') as file:
        file.write(texto_extraido)
    
    print("Texto extraído com sucesso e salvo no arquivo 'texto_extraido.txt'.")
else:
    print("Não foi possível extrair o texto da URL.")


In [ ]:

def preprocessar_texto(texto):
    # Tokenização usando o Tokenizador BERT (pode ser adaptado para outros tokenizadores)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokens = tokenizer.tokenize(texto)
    
    # Remoção de caracteres especiais e números
    tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens]
    
    # Remoção de stop words (exemplo com stop words em inglês)
    stop_words = set(["a", "an", "the", "in", "on", "at"])  # Adicione mais palavras conforme necessário
    tokens = [token for token in tokens if token.lower() not in stop_words]
    
    # Junção dos tokens de volta em uma string
    texto_processado = ' '.join(tokens)
    
    return texto_processado

In [ ]:
# with open('texto_extraido.txt') as file:
#     text = file.read()

In [ ]:
# processed = preprocessar_texto(text)

In [ ]:
# processed

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import numpy as np

# Carrega o modelo GPT-2 e o tokenizador
model_name = "gpt2"  # Pode ser "gpt2-medium", "gpt2-large", etc., para modelos maiores
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Texto de contexto
# contexto = "O aquecimento global é um fenômeno preocupante que está afetando o clima global."
# contexto = "Global warming is a worrisome phenomenon that is affecting the global climate."
contexto = "This Agenda is a plan of action for people, for the planet and for prosperity."

# Gera uma pergunta com base no contexto
input_text = f"Generate a question about: '{contexto}'"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Gera a pergunta
with torch.no_grad():
    resposta = model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=0)

# Decodifica e exibe a pergunta gerada
pergunta_gerada = tokenizer.decode(np.squeeze(resposta), skip_special_tokens=True)

print("Pergunta Gerada:")
print(pergunta_gerada)

In [ ]:
resposta

In [ ]:
#https://blog.ovhcloud.com/fine-tuning-llama-2-models-using-a-single-gpu-qlora-and-ai-notebooks/

In [ ]:
from transformers import AutoTokenizer, GPT2ForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2ForQuestionAnswering.from_pretrained("gpt2")

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"

inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]

# target is "nice puppet"
target_start_index = torch.tensor([14])
target_end_index = torch.tensor([15])

outputs = model(**inputs, start_positions=target_start_index, end_positions=target_end_index)
loss = outputs.loss

In [ ]:
outputs

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Carregar o modelo pré-treinado GPT-3.5 e o tokenizador
model_name = "EleutherAI/gpt-neo-2.7B"  # Este é um modelo GPT-3.5-like
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Definir o contexto
context = "O presidente dos Estados Unidos é Joe Biden. Ele foi eleito em 2020."

# Tokenizar o contexto
input_ids = tokenizer.encode(context, return_tensors="pt")

# Gerar perguntas
output = model.generate(input_ids, max_length=50, num_return_sequences=1, pad_token_id=50256)

# Decodificar a saída em texto legível
question = tokenizer.decode(output[0], skip_special_tokens=True)

print("Pergunta gerada: ", question)
